In [13]:
#Pip imports
#!pip install deep-translator

In [20]:
#imports
import numpy as np
import googletrans
from deep_translator import GoogleTranslator
import tritonclient.http as httpclient
from tritonclient.utils import np_to_triton_dtype

In [8]:
#Take wav file in and convert into text

In [22]:
#Input the wav file:
wavefile = "testing.wav"

In [23]:
#Using
def take_in_wav(wavefile):
    import riva.client
    auth = riva.client.Auth(uri='10.100.182.246:50051')
    asr_service = riva.client.ASRService(auth)

    offline_config = riva.client.RecognitionConfig(
        encoding=riva.client.AudioEncoding.LINEAR_PCM,
        max_alternatives=1,
        enable_automatic_punctuation=True,
        verbatim_transcripts=False,
    )

    my_wav_file = wavefile
    riva.client.add_audio_file_specs_to_config(offline_config, my_wav_file)

    with open(my_wav_file, 'rb') as fh:
        data = fh.read()

    response = asr_service.offline_recognize(data, offline_config)
    return response

In [24]:
def to_script(response):
    x = response.__str__().partition('transcript: ')
    x = x[2]
    x = x.__str__().partition('\n')
    y = x[0]
    return y

In [27]:
text = to_script(take_in_wav(speech), "en-US")

#Outputs a text file 

In [6]:
#Simplify it using the summarisation

In [28]:
#Parameters
URL = "10.98.96.143:8000"
MODEl_GPTJ_FASTERTRANSFORMER = "ensemble" 

OUTPUT_LEN = 128
BATCH_SIZE = 1
BEAM_WIDTH = 1
TOP_K = 20
TOP_P = 0.9

start_id = 220
end_id = 50256

In [34]:
client = httpclient.InferenceServerClient(URL, verbose=True)

In [35]:
client.get_server_metadata()

In [33]:
# Inference hyperparameters
def prepare_tensor(name, input):
    tensor = httpclient.InferInput(
        name, input.shape, np_to_triton_dtype(input.dtype))
    tensor.set_data_from_numpy(input)
    return tensor

# explanation
def prepare_inputs(input0):
    bad_words_list = np.array([["intelligence"]], dtype=object)
    stop_words_list = np.array([[""]], dtype=object)
    input0_data = np.array(input0).astype(object)
    output0_len = np.ones_like(input0).astype(np.uint32) * OUTPUT_LEN
    runtime_top_k = (TOP_K * np.ones([input0_data.shape[0], 1])).astype(np.uint32)
    runtime_top_p = TOP_P * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    beam_search_diversity_rate = 0.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    temperature = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    len_penalty = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    repetition_penalty = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    random_seed = 0 * np.ones([input0_data.shape[0], 1]).astype(np.int32)
    is_return_log_probs = True * np.ones([input0_data.shape[0], 1]).astype(bool)
    beam_width = (BEAM_WIDTH * np.ones([input0_data.shape[0], 1])).astype(np.uint32)
    start_ids = start_id * np.ones([input0_data.shape[0], 1]).astype(np.uint32)
    end_ids = end_id * np.ones([input0_data.shape[0], 1]).astype(np.uint32)

    inputs = [
        prepare_tensor("INPUT_0", input0_data),
        prepare_tensor("INPUT_1", output0_len),
        prepare_tensor("INPUT_2", bad_words_list),
        prepare_tensor("INPUT_3", stop_words_list),
        prepare_tensor("runtime_top_k", runtime_top_k),
        prepare_tensor("runtime_top_p", runtime_top_p),
        prepare_tensor("beam_search_diversity_rate", beam_search_diversity_rate),
        prepare_tensor("temperature", temperature),
        prepare_tensor("len_penalty", len_penalty),
        prepare_tensor("repetition_penalty", repetition_penalty),
        prepare_tensor("random_seed", random_seed),
        prepare_tensor("is_return_log_probs", is_return_log_probs),
        prepare_tensor("beam_width", beam_width),
        prepare_tensor("start_id", start_ids),
        prepare_tensor("end_id", end_ids),
    ]
    return inputs

In [ ]:
input0 = [[input_user],]
inputs = prepare_inputs(input0)
result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
output0 = result.as_numpy("OUTPUT_0")

In [ ]:
#Printing the summation
print(output0[0].decode('UTF-8'))